# Activity models for aqueous species

In [ ]:
from reaktoro import *
db = Database("supcrt98.xml")
editor = ChemicalEditor(db)
editor.addAqueousPhaseWithElements("H O C Na Cl Ca")
editor.addGaseousPhase(["H2O(g)"])

system = ChemicalSystem(editor)
print(system)

T = 25 # celsius
P = 1 # bar

problem1 = EquilibriumProblem(system)
problem1.setTemperature(T, "celsius")
problem1.setPressure(P, "bar")
problem1.add("H2O", 1, "kg")
problem1.add("NaCl", 1, "mol")

In [ ]:
state1 = equilibrate(problem1)
print(state1)

In [ ]:
problem2 = EquilibriumProblem(system)
problem2.setTemperature(T, "celsius")
problem2.setPressure(P, "bar")
problem2.add("H2O", 1, "kg")
problem2.add("CaCl2", 1, "mol")

In [ ]:
state2 = equilibrate(problem2)
print(state2)

In [ ]:
# Calculate mass of the solvent water
mw_H2O = 1000 / 18.0154 # where 18.0154 g/mol as an approximated molar mass of water
print(f"mw_H2O = {mw_H2O} kg / mol")

In [ ]:
# Collect system species and their amounts
species = system.species()
n = state2.speciesAmounts()

Fetching *indicies of aqueous species* and indices all species:

In [ ]:
indx_aqueous_phase = 0
indx_gaseous_phase = 1

indx_aq = system.indicesSpeciesInPhases([indx_aqueous_phase]) 
print("Indices of aquoues species:", indx_aq)
print("Indices of all species:", system.indicesSpeciesInPhases([indx_aqueous_phase, indx_gaseous_phase]) )

Collecting list of *aqueous species*, their names, and corresponding charges:

In [ ]:
species_aq = [species[i] for i in indx_aq]
n_aq = [n[i] for i in indx_aq]
names_aq = [species.name() for species in species_aq]
Z_aq = [species.charge() for species in species_aq]

In [ ]:
# Concetration of water
n_H2O = state2.speciesAmount("H2O(l)")

In [ ]:
import numpy as np # for divide function

# Molalities of aqueous species
m_aq = mw_H2O * np.divide(n_aq, n_H2O)
for name, Z, m in zip(names_aq, Z_aq, m_aq):
    print(f"{name:>10} : \t{Z:2.0f}, \t{m:6.2e} mol/kg")

Calculating ionic strength (IS):

In [ ]:
IS = 1/2 * sum([m * Z**2 for Z, m in zip(Z_aq, m_aq)])
print("Ionic strength = ", IS)

Calculation activity coefficients for aqueous ionic species using Davis model: 

In [ ]:
import math 
A_gamma = 0.5095
gamma = [10 ** (-A_gamma * Z**2 * (math.sqrt(IS)/(1 + math.sqrt(IS)) - 0.3 * IS)) for Z in Z_aq]

In [ ]:
for name, gamma_i in zip(names_aq, gamma):
    print(f"{name:>10} : \t{gamma_i:2.2f}")

> **Note**: In an ideal solution, $\gamma_i$ = 1 for all species. 

In [ ]:
A_gamma = 0.5095
I = np.linspace(0, 1, 100)

def gamma_davies(Zi, I):
    sqrtI = np.sqrt(I)
    return 10**(-A_gamma * Zi**2 * (sqrtI / (1.0 + sqrtI) - 0.3 * I))

# Coefficients
gamma_Z1 = gamma_davies(1.0, I)
gamma_Z2 = gamma_davies(2.0, I)
gamma_Z3 = gamma_davies(3.0, I)

In [ ]:
import matplotlib.pyplot as plt
plt.xlim((0, 1.0))
plt.xlabel('Ionic Strength [molal]')
plt.ylabel('Activity Coefficient')
line_Z1, = plt.plot(I, gamma_Z1, label=r'$Z_i=1$')
line_Z2, = plt.plot(I, gamma_Z2, label=r'$Z_i=2$')
line_Z3, = plt.plot(I, gamma_Z3, label=r'$Z_i=3$')
plt.legend(handles=[line_Z1, line_Z2, line_Z3], loc='upper right')
plt.show()
plt.savefig('activity-coefficient-davies.pdf')

Calculation of activity of the water solvent:

In [ ]:
# Collecting the species fractures
properties = state2.properties()
fractions = properties.moleFractions().val
for name, x in zip(names_aq, fractions):
    print(f"{name:>10} : \t{x:6.4e}")

In [ ]:
# Index of water in the chemical system
indx_H2O = system.indexSpecies("H2O(l)") 
print("Index of the water solvent is", indx_H2O)
print("Fraction of the water solvent is", fractions[indx_H2O])